# 1. Import Libraries

In [1]:
import cloudscraper
scraper = cloudscraper.create_scraper()
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import re
import timeit
from tqdm import tqdm
import time

# 2. AUTO TRADER

## 2.1. Scrape Car URLs

In [4]:
# 2.1. Scraping Car URLs - using Cloudscraper instead of Requests to circumvent cloudflare issues

# Empty lists to be filled by the following loops
car_URLs = []
car_makes = []

# All the required URLs:
# Base URL for car URLs to add onto below and URL with broad pre-refined search for all cars within parameters
urlBase = "https://www.autotrader.co.uk"
urlWebsite = "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2019&exclude-writeoff-categories=on&page=1"

# URLs for the year range divided evenly to overcome issue of max. scraping limit by Auto Trader 
urlWebsite_06_08 = "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2006&year-to=2008&exclude-writeoff-categories=on&page=1"
urlWebsite_09_11 = "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2009&year-to=2011&exclude-writeoff-categories=on&page=1"
urlWebsite_12_14 = "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2012&year-to=2014&exclude-writeoff-categories=on&page=1"
urlWebsite_15_17 = "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2015&year-to=2017&exclude-writeoff-categories=on&page=1"
urlWebsite_18_19 = "https://www.autotrader.co.uk/car-search?sort=relevance&postcode=dh14dg&radius=1500&include-delivery-option=on&price-from=1500&price-to=50000&year-from=2018&year-to=2019&exclude-writeoff-categories=on&page=1"

# Overall function to run the different sets of operations below
def scrape_all_URLs_and_makes(): 

    # Function for finding the next page
    def return_next_page(url):
        page = scraper.get(url).text
        soup = BeautifulSoup(page, "html.parser")
        return soup.find("a", class_="pagination--right__active")["href"] #finding the correct tags for the next page button

    # Function for finding all cars on page
    def return_next_car(url):
        page = scraper.get(url).text
        soup = BeautifulSoup(page, "html.parser")
        return soup.find_all("a", class_="js-click-handler listing-fpa-link tracking-standard-link") #finding correct tags for individual car URLs on the page

    # Function for finding all possible car makes within the broad refined search
    def get_car_makes(url):
        page = scraper.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        return soup.find("div", class_="sf-flyout__options js-flyout-options").text
    car_makes.append(get_car_makes(urlWebsite))
    
    # Function to run all of the following functions which get all car URLs
    def get_all_cars():
        if car_URLs not in car_URLs: # preventing duplicates --> note these still arise (not sure why), hence the list fromkeys in final part of cell code      
            # Getting cars between years 2006-2008
            def get_cars_06_08(urlWebsite_06_08):
                while (len(car_URLs) < 1285):
                    car_URLs.extend(urlBase + item["href"] for item in return_next_car(urlWebsite_06_08)) # extend the car URLs onto the list using the base URL
                    urlWebsite_06_08 = return_next_page(urlWebsite_06_08)

            # Getting cars between years 2009-2011
            def get_cars_09_11(urlWebsite_09_11):
                while (len(car_URLs) < 2570):
                    car_URLs.extend(urlBase + item["href"] for item in return_next_car(urlWebsite_09_11)) # extend the car URLs onto the list using the base URL
                    urlWebsite_09_11 = return_next_page(urlWebsite_09_11)

            # Getting cars between years 2012-2014
            def get_cars_12_14(urlWebsite_12_14):
                while (len(car_URLs) < 3855):
                    car_URLs.extend(urlBase + item["href"] for item in return_next_car(urlWebsite_12_14)) # extend the car URLs onto the list using the base URL
                    urlWebsite_12_14 = return_next_page(urlWebsite_12_14)

            # Getting cars between years 2015-2017
            def get_cars_15_17(urlWebsite_15_17):
                while (len(car_URLs) < 5140):
                    car_URLs.extend(urlBase + item["href"] for item in return_next_car(urlWebsite_15_17)) # extend the car URLs onto the list using the base URL
                    urlWebsite_15_17 = return_next_page(urlWebsite_15_17)

            # Getting cars between years 2018-2019
            def get_cars_18_19(urlWebsite_18_19):
                while (len(car_URLs) < 6425):
                    car_URLs.extend(urlBase + item["href"] for item in return_next_car(urlWebsite_18_19)) # extend the car URLs onto the list using the base URL
                    urlWebsite_18_19 = return_next_page(urlWebsite_18_19)        

            # Specifying the above functions 
            get_cars_06_08(urlWebsite_06_08)
            get_cars_09_11(urlWebsite_09_11)
            get_cars_12_14(urlWebsite_12_14)
            get_cars_15_17(urlWebsite_15_17)
            get_cars_18_19(urlWebsite_18_19)

    # Running the above functions
    get_car_makes(urlWebsite)
    get_all_cars()
                
scrape_all_URLs_and_makes()

# Checking the length of the list and a preview of the URLs to ensure it worked
car_URLs = list(dict.fromkeys(car_URLs)) # removes duplicates - ensures that  
print("Number of Car URLs: ")
print(len(car_URLs),"\n",) # Note the number of cars is over the number in the while loop above because it scrapes the entire page
print("Example Car URLs: ")
car_URLs[0], car_URLs[2000], car_URLs[3000], car_URLs[4500], car_URLs[6000]

Number of Car URLs: 
6354 

Example Car URLs: 


('https://www.autotrader.co.uk/car-details/202107175162349?price-from=1500&advertising-location=at_cars&exclude-writeoff-categories=on&sort=relevance&onesearchad=New&onesearchad=Nearly%20New&onesearchad=Used&price-to=50000&postcode=dh14dg&year-to=2008&include-delivery-option=on&radius=1500&year-from=2006&page=1',
 'https://www.autotrader.co.uk/car-details/202107265480974?advertising-location=at_cars&include-delivery-option=on&price-from=1500&onesearchad=New&onesearchad=Nearly%20New&onesearchad=Used&exclude-writeoff-categories=on&sort=relevance&year-from=2009&radius=1500&price-to=50000&postcode=dh14dg&year-to=2011&page=57',
 'https://www.autotrader.co.uk/car-details/202107124916105?price-from=1500&include-delivery-option=on&year-from=2012&onesearchad=New&onesearchad=Nearly%20New&onesearchad=Used&sort=relevance&radius=1500&advertising-location=at_cars&exclude-writeoff-categories=on&price-to=50000&postcode=dh14dg&year-to=2014&page=36',
 'https://www.autotrader.co.uk/car-details/2021072855

In [5]:
# Check for duplicates after the above
if len(car_URLs) == len(set(car_URLs)):
    print("No duplicate(s) present")
else:
    print("Duplicates present")

No duplicate(s) present


## 2.2. Scraping Specific Information From the Cars

In [6]:
# Function for automatically accepting cookies (a pop-up comes on the screen)
def remove_cookies():

    driver.switch_to.default_content() # Switch to default (must be done before switching frame)
    driver.switch_to.frame(driver.find_element_by_xpath("//iframe[@id='sp_message_iframe_441126']")) # Switch to frame
    driver.find_element_by_xpath("//button[@title='Accept All']").click() # Click the "Accept All" button for cookies
    driver.switch_to.default_content() # Switch back to default content

In [14]:
# driver.quit()

In [15]:
from selenium.webdriver.chrome.options import Options
options = Options ()
options.headless = True
driver = webdriver.Chrome('./chromedriver')#, options = options)

In [16]:
# Empty lists to be filled
make_model = []
year = []
mileage = []
transmission = []
engine_size = []
fuel_type = []
price = []
value_indicator = []

# Functions to get the correct information
# Presence of "try"/"except" so that "N/A" is written in an unattainable section and doesn't crash - will later be removed if N/A

# INCLUDE COMMENTS ON WHAT THE REGEX FUNCTIONS ARE DOING!

# Load the first car to accept all cookies before carrying out the scraping
driver.get(car_URLs[0])
try:
    time.sleep(5) # Wait to ensure the cookie pop-up has loaded 
    remove_cookies() # Carry out function which clicks the "Accept All" cookies button
    time.sleep(5) # Wait to ensure cookies have been accepted
except:
    print("COOKIES NOT ACCEPTED - RE-RUN CELL") # To alert if the cookies weren't accepted
    
# Loop which scrapes the relevant information from every car 
for car in car_URLs:
    driver.get(car)

    # VALUE - FIRST BECAUSE THIS IS THE MOST LIKELY TO NOT BE FILLED IN
    try:
        value_indicator.append(driver.find_element_by_class_name("price-indicator-gauge-container.price-indicator-gauge-container--dial").text)
    except:
        continue
        
    # MAKE AND MODEL
    try:
        make_model.append(driver.find_element_by_class_name("advert-heading__title.atc-type-insignia.atc-type-insignia--medium ").text)
    except:
        continue
    
    # YEAR - regex to keep just the year and not the registration year code too
    try:
        year.append(re.sub(r'\([^)]*\)', '', driver.find_element_by_class_name("advert-heading__year-title.atc-type-toledo").text))
    except:
        continue
    
    # MILEAGE - regex to remove all non-numbers 
    try:
        mileage.append(re.sub("\D", "", driver.find_element_by_class_name("mileage__distance").text))
    except:
        continue
    
    # TRANSMISSION
    try:
        transmission.append(driver.find_elements_by_xpath("//span[@class='price-confidence-key-specifications__text atc-type-fiesta']")[0].text)
    except:
        continue
    
    # ENGINE SIZE - regex to remove all letters (the presence of L)
    try:
        engine_size.append(re.sub('[A-z]*', "", driver.find_elements_by_xpath("//span[@class='price-confidence-key-specifications__text atc-type-fiesta']")[1].text))
    except:
        continue
     
    #FUEL TYPE
    try:
        fuel_type.append(driver.find_elements_by_xpath("//span[@class='price-confidence-key-specifications__text atc-type-fiesta']")[3].text)
    except:
        continue
    
    # PRICE - regex to remove all non-numbers
    try:
        price.append(re.sub("\D", "",driver.find_element_by_class_name("price-confidence__price").text))
    except:
        continue
    

In [327]:
# DataFrame Containing the Scraped Information

car_details = pd.DataFrame({"Make and Model": make_model, "Year":year, "Mileage":mileage, "Transmission": transmission, "Engine Size (L)":engine_size, "Fuel Type":fuel_type, "Price (£)":price, "Indicated Value":value_indicator})
car_details

,Make and Model,Year,Mileage,Transmission,Engine Size (L),Fuel Type,Price (£),Indicated Value
0,Vauxhall Astra,2007,72000,Manual,1.6,Petrol,1999,Fair price
1,Renault Clio,2007,65000,Manual,1.2,Petrol,1550,Great price
2,BMW 3 Series,2006,81500,Manual,2.0,Petrol,1990,Lower price
3,Renault Clio,2006,55378,Manual,1.6,Petrol,1675,Lower price
4,Honda Civic,2008,121000,Manual,1.8,Petrol,1995,Good price
...,...,...,...,...,...,...,...,...
5805,Nissan Micra,2018,8137,Manual,0.9,Petrol,9410,Great price
5806,Peugeot 2008,2018,17356,Manual,1.6,Diesel,10995,Fair price
5807,Vauxhall Corsa,2019,6500,Manual,1.4,Petrol,9795,Fair price
5808,Citroen C1,2018,43000,Manual,1.0,Petrol,5995,Great price


In [328]:
car_details.count()

# However, as we will see below, this does not show the full story and masks faults - thus should not be used...
# as a way of confirming everything is correct

Make and Model     5810
Year               5810
Mileage            5810
Transmission       5810
Engine Size (L)    5810
Fuel Type          5810
Price (£)          5810
Indicated Value    5810
dtype: int64

## 2.3. Cleaning the data

In [376]:
# Create a copy for the following cleaning
AT_details = car_details.copy(deep=True)

##### a. Dropping Duplicates

In [378]:
# Duplicates can be present due to sellers paying more to advertise their car more (sponsored ad's)
# HOW DO THESE ARISE IF THERE ARE NO DUPLICATE URLS?! WEBSITE SET UP IN SUCH A WAY THAT IT DOES THIS FOR SCRAPING?

# checking that there are duplicates
print(AT_details.value_counts()[0:5], "\n") 

# drop duplicates and see updated dataframe
AT_details = AT_details.drop_duplicates()

# ensure duplicates have been removed
print(AT_details.value_counts()[0:5], "\n") 

# check the length of the new dataframe
print("Length of dataframe without duplicates:", len(AT_details))
print("Number of duplicates dropped:", (len(car_details)-len(AT_details)))

Make and Model         Year   Mileage  Transmission  Engine Size (L)  Fuel Type  Price (£)  Indicated Value
Vauxhall Astra         2007   72000    Manual        1.6              Petrol     1999       Fair price         42
Volkswagen Polo        2008   70598    Manual        1.2              Petrol     2996       Fair price         36
Mercedes-Benz A Class  2007   91400    Manual        1.5              Petrol     1999       Fair price         34
Vauxhall Astra         2006   75000    Manual        1.4              Petrol     1799       Fair price         34
MG Mg3                 2016   35361    Manual        1.5              Petrol     5250       Good price         27
dtype: int64 

Make and Model  Year   Mileage  Transmission  Engine Size (L)  Fuel Type  Price (£)  Indicated Value
Volvo XC70      2006   146562   Automatic     2.4              Diesel     1800       Lower price        1
Hyundai I30     2018   34365    Manual        1.0              Petrol     10500      Great price    

##### b. Removing Blank Cells Which Arise with Electric Vehicles and Sponsored Advertisements which load cars outside the set parameters

In [379]:
# However, blanks arise when the car is electric. This is because the layout of the "key specs" section is...
# different for these cars because it has no engine size input (e.g. it has the number of doors where the fuel...
# type normally is), and thus when my scraper scrapes them and runs the regex, it deletes these entries... 
# leaving them blank, and therefore all we can do for now is remove them from the DF
# Blank cells also arise when sponsored ad's cause the scraping of cars are outside of the set parameters (e.g...
# too old and therefore do not have an indicated value

# Show the rows which had a blank cell and fill those cells with "NaN"

AT_details = AT_details.replace('', np.nan) # Replace empty cells with "NaN"
nan_values = AT_details[AT_details.isna().any(axis=1)] # Get rows with "NaN" values
nan_values # Show the rows with "NaN" values

,Make and Model,Year,Mileage,Transmission,Engine Size (L),Fuel Type,Price (£),Indicated Value
46,Vauxhall Corsa,2008,36000,Manual,1.2,Petrol,1995,NaN
5347,Nissan Leaf,2018,71260,Automatic,NaN,5 doors,13175,Good price


In [380]:
# Drop the cells which had NaN and reset the index 

AT_details.dropna(axis = 0, how = "any", inplace=True)
AT_details = AT_details.reset_index(drop=True)
AT_details

,Make and Model,Year,Mileage,Transmission,Engine Size (L),Fuel Type,Price (£),Indicated Value
0,Vauxhall Astra,2007,72000,Manual,1.6,Petrol,1999,Fair price
1,Renault Clio,2007,65000,Manual,1.2,Petrol,1550,Great price
2,BMW 3 Series,2006,81500,Manual,2.0,Petrol,1990,Lower price
3,Renault Clio,2006,55378,Manual,1.6,Petrol,1675,Lower price
4,Honda Civic,2008,121000,Manual,1.8,Petrol,1995,Good price
...,...,...,...,...,...,...,...,...
4560,SKODA Octavia,2018,72681,Manual,1.6,Diesel,10100,Good price
4561,Peugeot 108,2019,6323,Automatic,1.0,Petrol,9995,Good price
4562,Nissan Micra,2018,8137,Manual,0.9,Petrol,9410,Great price
4563,Peugeot 2008,2018,17356,Manual,1.6,Diesel,10995,Fair price


In [381]:
# The above happens only for electric cars because their key spec format is different 

blank_present = AT_details.isin([""]).any().any()
if blank_present:
    print("Blank cell(s) present")
else:
    print("No blank cell(s) present")

No blank cell(s) present


##### c. Keeping just petrol and diesel cars - removes hybrids as there are too few of them for accurate modelling and electric vehicles which have been scraped incorrectly 

In [382]:
# Show the different fuel types scraped and their quantities
print(AT_details.value_counts("Fuel Type")) 

Fuel Type
Petrol                   3037
Diesel                   1509
Petrol Hybrid              14
Petrol Plug-in Hybrid       4
Diesel Hybrid               1
dtype: int64


In [386]:
# Keep only petrol and diesel cars
fuel_keep = "^(Petrol|Diesel)$" # keep only occurences of petrol or diesel
filter = AT_details['Fuel Type'].str.contains(fuel_keep) # filter the fuel type column of the dataframe for the correct fuel types
AT_details = AT_details[filter] # apply the filter to the dataframe

print(AT_details.value_counts("Fuel Type")) # check to make sure the above has worked
AT_details = AT_details.reset_index(drop=True) # reset the index 
AT_details # check it has worked

Fuel Type
Petrol    3037
Diesel    1509
dtype: int64


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,Make and Model,Year,Mileage,Transmission,Engine Size (L),Fuel Type,Price (£),Indicated Value
0,Vauxhall Astra,2007,72000,Manual,1.6,Petrol,1999,Fair price
1,Renault Clio,2007,65000,Manual,1.2,Petrol,1550,Great price
2,BMW 3 Series,2006,81500,Manual,2.0,Petrol,1990,Lower price
3,Renault Clio,2006,55378,Manual,1.6,Petrol,1675,Lower price
4,Honda Civic,2008,121000,Manual,1.8,Petrol,1995,Good price
...,...,...,...,...,...,...,...,...
4541,SKODA Octavia,2018,72681,Manual,1.6,Diesel,10100,Good price
4542,Peugeot 108,2019,6323,Automatic,1.0,Petrol,9995,Good price
4543,Nissan Micra,2018,8137,Manual,0.9,Petrol,9410,Great price
4544,Peugeot 2008,2018,17356,Manual,1.6,Diesel,10995,Fair price


##### d. Finding Car Makes and Splitting Makes and Models Into Separate Columns Accordingly

In [366]:
# Clean the list of makes

# iterate through the uncleaned list of car_makes and split string whenever there was an "\n"
for i, item in enumerate(car_makes):
    car_makes_clean = ' '.join(item.split("\n"))

# remove any non-letters
car_makes_clean = re.sub(r'[^A-z -]+', '', car_makes_clean)

# strip leading and trailing whitespaces and make lowercase
car_makes_clean = car_makes_clean.strip().lower()

#split at three spaces to get a list of the individual makes
car_makes_clean = re.split("   ", car_makes_clean)
print(car_makes_clean)

['abarth', 'aixam', 'ak', 'alfa romeo', 'alpine', 'ariel', 'aston martin', 'audi', 'austin', 'beauford', 'bentley', 'bmw', 'bramwith', 'cadillac', 'caterham', 'chesil', 'chevrolet', 'chrysler', 'citroen', 'corvette', 'cupra', 'dacia', 'daihatsu', 'daimler', 'dax', 'dodge', 'ds automobiles', 'fiat', 'ford', 'fso', 'ginetta', 'gmc', 'great wall', 'holden', 'honda', 'hummer', 'hyundai', 'infiniti', 'isuzu', 'iveco', 'jaguar', 'jeep', 'kia', 'lancia', 'land rover', 'levc', 'lexus', 'lincoln', 'london taxis international', 'lotus', 'mahindra', 'maserati', 'mazda', 'mercedes-benz', 'mg', 'microcar', 'mini', 'mitsubishi', 'morgan', 'nissan', 'opel', 'perodua', 'peugeot', 'porsche', 'proton', 'rage', 'renault', 'reva', 'rover', 'saab', 'santana', 'seat', 'sebring', 'skoda', 'smart', 'ssangyong', 'subaru', 'suzuki', 'tesla', 'tiger', 'toyota', 'tvr', 'ultima', 'vauxhall', 'volkswagen', 'volvo', 'westfield', 'yamaha', 'zenos']


In [388]:
# Make lowercase
AT_details["Make and Model"] = AT_details["Make and Model"].str.lower()
AT_details # check to see it has worked 

,Make and Model,Year,Mileage,Transmission,Engine Size (L),Fuel Type,Price (£),Indicated Value
0,vauxhall astra,2007,72000,Manual,1.6,Petrol,1999,Fair price
1,renault clio,2007,65000,Manual,1.2,Petrol,1550,Great price
2,bmw 3 series,2006,81500,Manual,2.0,Petrol,1990,Lower price
3,renault clio,2006,55378,Manual,1.6,Petrol,1675,Lower price
4,honda civic,2008,121000,Manual,1.8,Petrol,1995,Good price
...,...,...,...,...,...,...,...,...
4541,skoda octavia,2018,72681,Manual,1.6,Diesel,10100,Good price
4542,peugeot 108,2019,6323,Automatic,1.0,Petrol,9995,Good price
4543,nissan micra,2018,8137,Manual,0.9,Petrol,9410,Great price
4544,peugeot 2008,2018,17356,Manual,1.6,Diesel,10995,Fair price


In [391]:

make_model_test = list(AT_details["Make and Model"])

clean_make_list = []
clean_model_list = []

for car in AT_details["Make and Model"]:
    for make in car_makes_clean:
        if make in car:
            clean_make_list.append(make)
            clean_model_list.append(car[len(make):].strip())
            break

AT_details["Make"] = clean_make_list
AT_details["Model"] = clean_model_list

AT_details

,Make and Model,Year,Mileage,Transmission,Engine Size (L),Fuel Type,Price (£),Indicated Value,Make,Model
0,vauxhall astra,2007,72000,Manual,1.6,Petrol,1999,Fair price,vauxhall,astra
1,renault clio,2007,65000,Manual,1.2,Petrol,1550,Great price,renault,clio
2,bmw 3 series,2006,81500,Manual,2.0,Petrol,1990,Lower price,bmw,3 series
3,renault clio,2006,55378,Manual,1.6,Petrol,1675,Lower price,renault,clio
4,honda civic,2008,121000,Manual,1.8,Petrol,1995,Good price,honda,civic
...,...,...,...,...,...,...,...,...,...,...
4541,skoda octavia,2018,72681,Manual,1.6,Diesel,10100,Good price,skoda,octavia
4542,peugeot 108,2019,6323,Automatic,1.0,Petrol,9995,Good price,peugeot,108
4543,nissan micra,2018,8137,Manual,0.9,Petrol,9410,Great price,nissan,micra
4544,peugeot 2008,2018,17356,Manual,1.6,Diesel,10995,Fair price,peugeot,2008


## Save as CSV

In [392]:
AT_details.to_csv("Auto Trader Cars.csv")